In [ ]:
import os 
import pickle
import pandas as pd
import numpy as np
from importlib import reload

import feature_selection
import sampling

reload(feature_selection)
reload(sampling)

from feature_selection import select_common_features
from sampling import create_stratified_kfolds, create_stratified_train_test_sets


from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, \
    matthews_corrcoef, mean_squared_error, r2_score, roc_auc_score, roc_curve, auc
from math import sqrt
from sklearn.model_selection import GridSearchCV

from summarytools import dfSummary
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.pipeline import Pipeline as Pipeline_imb
from sklearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler, TomekLinks

# Impute missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer

%matplotlib inline

In [ ]:
# TEMPORARY
import warnings
warnings.filterwarnings("ignore")

### Notebook parameters

In [ ]:
# notebook parameters
input_dataset_path = "data/heart_disease_health_indicators_BRFSS2015.csv"
target_col = "HeartDiseaseorAttack"
separator = ','

generate_new_folds = False
n_splits = 10

In [ ]:
fix_imbalanced_dataset = True
use_oversampling = True
use_undersampling = False

In [ ]:
imputing_method = 'mean'

In [ ]:
models_saving_directory = 'models/2SMOTE_no_imputer_15pct'
datasets_with_nans_dir = 'missing_values_folds_15pct'
datasets_with_imputed_values_dir = 'missing_values_folds_15pct/median_imputer'

### Functions

In [ ]:
def calculate_metrics(y_pred: np.array, y_test: pd.Series):
    """ Calculate model quality metrics based on 
        expected label values from testing dataset (y_test) and predicted values.
    """
    tn, fp, fn, tp = calculate_test_results_from_confusion_matrix(y_test, y_pred)
    model_precision = precision_score(y_test, y_pred)
    model_recall = recall_score(y_test, y_pred)
    model_specificity = specificity_score(tn, fp)
    model_acc = accuracy_score(y_test, y_pred)
    model_npv = calculate_npv(tn, fn)

    model_f1_score = f1_score(y_test, y_pred)
    model_mcc = matthews_corrcoef(y_test, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    model_r2 = r2_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    model_scores = {
        "True_Negative": tn,
        "False_Positive": fp,
        "False_Negative": fn,
        "True_Positive": tp,
        "Precision_PPV": model_precision,
        "Sensitivity_TPR_Recall": model_recall,
        "Speciticity_TNR": model_specificity,
        "Accuracy": model_acc,
        "Negative_Predictive_Value_NPV": model_npv,
        "F1_Score": model_f1_score,
        "RMSE": rmse,
        "R_Squared": model_r2,
        "Matthews_Correlation_Coefficient_MCC": model_mcc,
        "ROC_AUC_score": roc_auc
    }

    model_scores_df = pd.DataFrame(model_scores.values(), index=model_scores.keys()).transpose()

    return model_scores_df

def calculate_test_results_from_confusion_matrix(y_test: pd.DataFrame, y_pred: pd.DataFrame):
    """ Calculate the confusion matrix and extract TP, FP, TN, FN from that matrix """
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    return tn, fp, fn, tp

def specificity_score(tn: float, fp: float):
    return tn / (tn + fp)

def calculate_npv(tn: float, fn: float):
    return tn / (tn + fn)

### Load data

In [ ]:
# Read folds that are available
# Create empty lists to store train and test DataFrames
train_datasets = []
test_datasets = []

for fold_num in range(1, n_splits+1):
    train_file_path = f"{datasets_with_nans_dir}/fold_{fold_num}_train.csv"
    test_file_path = f"{datasets_with_nans_dir}/fold_{fold_num}_test.csv"
    
    # Load the train and test fold data into DataFrames
    train_fold = pd.read_csv(train_file_path)
    test_fold = pd.read_csv(test_file_path)
    
    train_datasets.append(train_fold)
    test_datasets.append(test_fold)
print("Folds data were loaded successfully!")

In [ ]:
best_features = ['GenHlth', 'PhysHlth', 'Age', 'HighChol', 'Diabetes', 'Income', 'HighBP']

In [ ]:
if imputing_method == 'mean' or imputing_method == 'median':
    imputer_obj = SimpleImputer(strategy=imputing_method)
elif imputing_method == 'knn':
    imputer_obj = KNNImputer(n_neighbors=3)
else:
    imputer_obj = IterativeImputer()

In [ ]:
train_data_copy = []
for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):
    train_fold_copy = train_data.copy(deep=True)
    train_data_copy.append(train_fold_copy)
    train_data = imputer_obj.fit_transform(train_data)

    # Save each fold as a CSV file
    pd.DataFrame(train_data, columns=train_fold_copy.columns).to_csv(f'{datasets_with_imputed_values_dir}/fold_{fold_num}_train.csv', index=False)
    test_data.to_csv(f'{datasets_with_imputed_values_dir}/fold_{fold_num}_test.csv', index=False)

In [ ]:
# Read folds that are available
# Create empty lists to store train and test DataFrames
train_datasets = []
test_datasets = []

for fold_num in range(1, n_splits+1):
    train_file_path = f"{datasets_with_imputed_values_dir}/fold_{fold_num}_train.csv"
    test_file_path = f"{datasets_with_imputed_values_dir}/fold_{fold_num}_test.csv"
    
    # Load the train and test fold data into DataFrames
    train_fold = pd.read_csv(train_file_path)
    test_fold = pd.read_csv(test_file_path)
    
    train_datasets.append(train_fold)
    test_datasets.append(test_fold)
print("Folds data were loaded successfully!")

In [ ]:
clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=1)
clflr2 = LogisticRegression()
clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
clf3 = DecisionTreeClassifier(criterion='gini', random_state=1)
clf4 = SVC(kernel="linear", C=0.3, random_state=1)
clf5 = RandomForestClassifier(random_state=1)
clf6 = XGBClassifier(objective= 'binary:logistic',
                    nthread=4,
                    seed=42)
clf7 = GaussianNB()
# clf8 = SVC(gamma='scale', class_weight='balanced')
clf9 = LinearSVC(class_weight='balanced')
clf10 = DecisionTreeClassifier(criterion='entropy', random_state=42, class_weight='balanced')
clf11 = RandomForestClassifier(random_state=1, class_weight='balanced')
clf12 = LogisticRegression(class_weight='balanced', random_state=42)
clf13 = ComplementNB(force_alpha=True)

In [ ]:
if fix_imbalanced_dataset and use_oversampling:
    sampler = SMOTE(random_state=42)
    # sampler = RandomOverSampler(random_state=42)
elif fix_imbalanced_dataset and use_undersampling:
    # sampler = TomekLinks()
    sampler = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
    # sampler = RandomUnderSampler(random_state=42)

In [ ]:
if fix_imbalanced_dataset:
    pipe1 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('LR', clf1)])
    
    pipelr2 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('LR2', clflr2)])

    pipe2 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('KNN', clf2)])
    
    pipe3 = Pipeline_imb([('sampler', sampler),
                          ('DT', clf3)])

    pipe4 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('SVM', clf4)])
    
    pipe5 = Pipeline_imb([('sampler', sampler),
                          ('RF', clf5)])

    pipe6 = Pipeline_imb([('sampler', sampler),
                          ('XGB', clf6)])

    pipe7 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('GNB', clf7)])
    
    # pipe8 = Pipeline_imb([('sampler', sampler),
    #                       ('scaler', StandardScaler()),
    #                       ('bSVC', clf8)])
    
    pipe9 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('LSCV', clf9)])
    
    pipe10 = Pipeline_imb([('sampler', sampler),
                          ('bDT', clf10)])
    
    pipe11 = Pipeline_imb([('sampler', sampler),
                          ('bRF', clf11)])
    
    pipe12 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('bLR', clf12)])
    
    pipe13 = Pipeline_imb([('sampler', sampler),
                          ('scaler', StandardScaler()),
                          ('CNB', clf13)])

else:
    # Building the pipelines based on pre defined classifiers
    pipe1 = Pipeline([('scaler', StandardScaler()),
                    ('LR', clf1)])
    
    pipelr2 = Pipeline([('scaler', StandardScaler()),
                          ('LR2', clflr2)])

    pipe2 = Pipeline([('scaler', StandardScaler()),
                    ('KNN', clf2)])
    
    pipe3 = Pipeline([('DT', clf3)])

    pipe4 = Pipeline([('scaler', StandardScaler()),
                    ('SVM', clf4)])
    
    pipe5 = Pipeline([('RF', clf5)])

    pipe6 = Pipeline([('XGB', clf6)])

    pipe7 = Pipeline([('scaler', StandardScaler()),
                    ('GNB', clf7)])
    
    # pipe8 = Pipeline([('scaler', StandardScaler()),
    #                       ('bSVC', clf8)])
    
    pipe9 = Pipeline([('scaler', StandardScaler()),
                          ('LSCV', clf9)])
    
    pipe10 = Pipeline([('bDT', clf10)])
    
    pipe11 = Pipeline([('bRF', clf11)])
    
    pipe12 = Pipeline([('scaler', StandardScaler()),
                          ('bLR', clf12)])
    
    pipe13 = Pipeline([('scaler', MinMaxScaler()),
                       ('CNB', clf13)])


In [ ]:
model_results_df_lst = []

for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):

    # # ########## Take 2:1 ratio of negative to positive class #########################

    # positive_class_df = train_data[train_data[target_col]==1]
    # negative_class_df = train_data[train_data[target_col]==0]

    # positive_class_samples_count = len(positive_class_df)
    # negative_class_samples_count = 2*positive_class_samples_count

    # randomly_selected_negative_class_samples = negative_class_df.sample(n=negative_class_samples_count, random_state=42)
    
    # preselected_train_data = pd.concat([positive_class_df, randomly_selected_negative_class_samples], axis=0)
    # preselected_train_data = preselected_train_data.sample(frac=1.0, ignore_index=True)

    # # ################################################################################

    fX_train = train_data.drop(columns=[target_col])[best_features]
    fy_train = train_data[target_col]

    fX_test = test_data.drop(columns=[target_col])[best_features]
    fy_test = test_data[target_col]

    fold_results = []

    for est, name in zip((pipe1, pipelr2, pipe3, pipe5, pipe6, pipe7, pipe2, pipe9, pipe10, pipe11, pipe12), ('LR', 'LR2', 'DT', 'RF', 'XGB', 'GNB', 'KNN', 'LSCV', 'bDT', 'bRF', 'bLR')):
        est.fit(X=fX_train, y=fy_train)
        fy_pred = est.predict(X=fX_test)

        # Create a directory if it doesn't exist
        if not os.path.exists(f'{models_saving_directory}/{name}'):
            os.makedirs(f'{models_saving_directory}/{name}')

        with open(f'{models_saving_directory}/{name}/{name}_model_fold_no{fold_num}.pickle', 'wb') as file:
            pickle.dump(est.named_steps[name], file)

        est_scores_df = calculate_metrics(y_test=fy_test, y_pred=fy_pred)

        fpr, tpr, thresholds = roc_curve(fy_test, fy_pred)
        roc_auc = roc_auc_score(fy_test, fy_pred)

        # Plot the ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Receiver Operating Characteristic (ROC) Curve for {name}')
        plt.legend(loc='lower right')
        plt.show()

        est_scores_df['model_name'] = name
        est_scores_df.to_excel(f'{models_saving_directory}/{name}/{name}_model_result_fold_no{fold_num}.xlsx')
        print(f'Log: {name} finished')

        fold_results.append(est_scores_df)

    fold_results_df = pd.concat(fold_results, ignore_index=True)
    fold_results_df['fold no'] = fold_num

    model_results_df_lst.append(fold_results_df)
    
    print(f'Fold no: {fold_num} finished')
    print(50 * '-', '\n')

fs_model_results_df = pd.concat(model_results_df_lst, ignore_index=True)

In [ ]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
fs_model_results_df.to_excel(f'{models_saving_directory}/models_results_{timestamp}.xlsx', header=True)

In [ ]:
with open(f'{models_saving_directory}/used_features_{timestamp}.txt', "w") as file:
    for item in fX_train.columns:
        file.write(str(item) + "\n")

In [ ]:
fs_model_results_df

### Generate ROC curve for LR, GNB and XGB

In [ ]:
model_results_df_lst = []

# Create empty lists to store ROC data for each fold
all_fpr = []
all_tpr = []
all_thresholds = []

for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):

    fX_train = train_data.drop(columns=[target_col])[best_features]
    fy_train = train_data[target_col]

    fX_test = test_data.drop(columns=[target_col])[best_features]
    fy_test = test_data[target_col]

    fold_results = []

    for est, name in zip((pipe1, pipe6, pipe7), ('LR', 'XGB', 'GNB')):
        est.fit(X=fX_train, y=fy_train)
        fy_pred = est.predict_proba(X=fX_test)
        # Extract the probability estimates for the positive class (class 1)
        y_probabilities_positive = fy_pred[:, 1]

        # Create a directory if it doesn't exist
        if not os.path.exists(f'{models_saving_directory}/{name}'):
            os.makedirs(f'{models_saving_directory}/{name}')

        with open(f'{models_saving_directory}/{name}/{name}_model_fold_no{fold_num}.pickle', 'wb') as file:
            pickle.dump(est.named_steps[name], file)

        if name == 'GNB':
            fpr, tpr, thresholds = roc_curve(fy_test, y_probabilities_positive)
            roc_auc = roc_auc_score(fy_test, y_probabilities_positive)

            all_fpr.append(fpr)
            all_tpr.append(tpr)
            all_thresholds.append(thresholds)

        print(f'Log: {name} finished')
    
    print(f'Fold no: {fold_num} finished')
    print(50 * '-', '\n')

# Combine all thresholds from different folds
combined_thresholds = [item for sublist in all_thresholds for item in sublist]

# Plot the ROC curve
plt.figure(figsize=(8, 6))
for i in range(len(all_fpr)):
    plt.plot(all_fpr[i], all_tpr[i], label=f'Fold {i + 1}', alpha=0.5)

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Krzywa ROC dla 10-foldów')
plt.legend(loc='lower right')